In [ ]:
import numpy as np

from scipy.special import lambertw

import sympy

from regions import CirclePixelRegion, PixCoord, RectanglePixelRegion, CirclePixelRegion

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

See [exp_gen_experiments](exp_gen_experiments.ipynb) for function tests and derivations

In [ ]:
BVAL = 1.6783469900166605 
TWOPI = 2*np.pi

In [ ]:
def generate_unif_box(n=None, halfsize=1, density=None, rstate=np.random.RandomState()):
    if n is not None and density is not None:
        raise ValueError('cannot give both n and density')
    elif density is not None:
        A = (halfsize*2)**2
        n = density * A
        
    if n != int(n):
        remainder = n - int(n)
        n = int(n)
        if np.random.rand(1)[0] < remainder:
            n += 1
    else:
        n = int(n)
        
    x = rstate.uniform(-halfsize, halfsize, n)
    y = rstate.uniform(-halfsize, halfsize, n)

    return x, y

def uniform_in_region(density, region, rstate=np.random.RandomState()):
    maxpx = np.max(region.bounding_box.extent)
    xs, ys = generate_unif_box(halfsize=maxpx, density=density, rstate=rstate)
    msk = region.contains(PixCoord(xs, ys))

    return xs[msk], ys[msk]

def expinteg(F, Re, b=bval):
    sc = np.isscalar(F)
    res = np.atleast_1d(-Re*(lambertw((F-1)/np.exp(1), -1).real + 1)/b)
    res.ravel()[np.isnan(res.ravel())] = 0
    return res[0] if sc else res
    
def generate_exp_dglx(n=None, Re=1, density=None, region=None, rstate=np.random.RandomState()):
    if n is not None and density is not None:
        raise ValueError('cannot give both n and density')
    elif density is not None:
        # density is the mean density within the HLR
        hA = np.pi*Re**2
        n = 2* density * hA
        
    if n != int(n):
        remainder = n - int(n)
        n = int(n)
        if np.random.rand(1)[0] < remainder:
            n += 1
    else:
        n = int(n)
        
    U = rstate.uniform(0, 1, n)
    r = expinteg(U, Re)
    phi = rstate.uniform(0, 2*np.pi, n)

    x = np.cos(phi)*r
    y = np.sin(phi)*r

    if region is not None:
        msk = region.contains(PixCoord(x, y))
        x = x[msk]
        y = y[msk]

    return x, y

In [ ]:
def glx_and_uniform(glx_hdensity, bkg_density, region, Re=1, rstate=np.random.RandomState()):
    g = generate_exp_dglx(density=glx_hdensity, Re=Re, region=region, rstate=rstate)
    b = uniform_in_region(bkg_density, region=region, rstate=rstate)

    return np.concatenate((g, b), axis=1)

In [ ]:
circular_large_reg = CirclePixelRegion(PixCoord(0, 0), 10)
circular_small_reg = CirclePixelRegion(PixCoord(0, 0), 3)
rectangular_large_reg = RectanglePixelRegion(PixCoord(0, 0), 20, 20)
rectangular_small_reg = RectanglePixelRegion(PixCoord(0, 0), 6, 6)

offset_region1 = RectanglePixelRegion(PixCoord(2, 3), 2, 3)
offset_region2 = RectanglePixelRegion(PixCoord(0, -1), 6, 6)
offset_regions = offset_region1 | offset_region2

all_regions = {nm: globals()[nm] for nm in 'circular_large_reg,circular_small_reg,rectangular_large_reg,rectangular_small_reg,offset_regions'.split(',')}

# Set up fitting code

`region` is a fixed region - if None the integrals are implied to infity.

Parameters:
* x0, y0 -> center
* Re -> half-light radius
* fbkg -> background fraction of all the stars

In [ ]:
import dynesty
from dynesty import plotting as dyplot

param_names = 'x0, y0, Re, fbkg'.split(', ')

Derive the bounded scale-free prior generator

In [ ]:
σ, x, a, b, A, F = sympy.symbols('σ,x,a,b,A,F', real=True, positive=True)
Asoln = sympy.solve(sympy.integrate(A/σ,(σ,a,b)) - 1, A)[0]
sympy.solve(sympy.integrate(A/σ,(σ,a,x)).subs(A, Asoln) - F, x)[0]

Prior below is that x0, y0 are on U[-1, 1], Re is scale-free on [.01, 100], fbkg is U[.1, .9]

In [ ]:
uppri = 100
lpri = .01

def prior_transform(u):
    x = np.empty_like(u)
    x[:2] = u[:2]*2 -1
    x[2] = lpri**(1-u[2]) * uppri**u[2]
    x[3] = u[3]*.8 + .1

    return x

In [ ]:
LTWOPI = np.log(TWOPI)

def loglike(p, data):
    x0, y0, Re, fbkg = p
    x, y = data
    α = Re/BVAL

    r = np.hypot(x-x0, y-y0)
    lgal = np.log(1-fbkg) - 2*np.log(α) - r/α - LTWOPI
    lbkg = np.log(fbkg)
    ll = np.logaddexp(lgal, lbkg)

    return np.sum(ll)
    

In [ ]:
dsampler = dynesty.DynamicNestedSampler(loglike, prior_transform, len(param_names),
                                       logl_kwargs={'data':glx_and_uniform(100, 10, circular_large_reg)},
                                       sample='rslice')
truths = [0, 0, 1, 100/10]
dsampler.run_nested(maxiter=100000,  wt_kwargs={'pfrac': 1.0})

In [ ]:
fig, axes = dyplot.runplot(dsampler.results)
fig, axes = dyplot.traceplot(dsampler.results, labels=param_names, truths=truths)
fig, axes = dyplot.cornerpoints(dsampler.results, labels=param_names, truths=truths)